# Import

In [1]:
import csv
import numpy as np
np.random.seed(0)
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)
import pickle
from bs4 import BeautifulSoup
from nltk.stem.snowball import SnowballStemmer
from string import digits
import re
import string
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from gensim.models import Word2Vec
import nltk

Using TensorFlow backend.
C:\Users\rayha\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


## loading data

In [2]:
train_data = pickle.load(open( "../ML classification/training_data.pkl", "rb"))
test_data = pickle.load(open( "../ML classification/testing_data.pkl", "rb"))

In [3]:
train_data.head()

,text_data,target_label
0,From: mathew <mathew@mantis.co.uk>\nSubject: A...,alt.atheism
1,From: mathew <mathew@mantis.co.uk>\nSubject: A...,alt.atheism
2,From: I3150101@dbstu1.rz.tu-bs.de (Benedikt Ro...,alt.atheism
3,From: mathew <mathew@mantis.co.uk>\nSubject: R...,alt.atheism
4,From: strom@Watson.Ibm.Com (Rob Strom)\nSubjec...,alt.atheism


In [4]:
test_data.head()

,text_data,target_label
0,From: decay@cbnewsj.cb.att.com (dean.kaflowitz...,alt.atheism
1,From: cfaehl@vesta.unm.edu (Chris Faehl)\nSubj...,alt.atheism
2,From: mathew <mathew@mantis.co.uk>\nSubject: R...,alt.atheism
3,"From: dps@nasa.kodak.com (Dan Schaertel,,,)\nS...",alt.atheism
4,From: halat@panther.bears (Jim Halat)\nSubject...,alt.atheism


In [5]:
labels = list(train_data["target_label"].unique())

In [6]:
Y_train = []
for y in train_data["target_label"]:
    Y_train.append(labels.index(y))

In [7]:
Y_test = []
for y in test_data["target_label"]:
    Y_test.append(labels.index(y))

In [8]:
Y_train[-10:]

[19, 19, 19, 19, 19, 19, 19, 19, 19, 19]

In [9]:
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

In [10]:
def clean_text(text):
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    
    ##remove numbers
    remove_digits = str.maketrans('', '', digits)
    text = text.translate(remove_digits)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"cannot", "can not ", text)
    text = re.sub(r"gonna", "going to ", text)
    text = re.sub(r"gotta", "got to ", text)
    text = re.sub(r"wanna", "want to ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ", text)
    text = re.sub(r";", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"\<", " ", text)
    #text = text.replace("paraphernalia", "")
    
    
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)

    return text

In [11]:
train_data['text_data'] = train_data['text_data'].map(lambda x: clean_text(x))

In [12]:
test_data['text_data'] = test_data['text_data'].map(lambda x: clean_text(x))

In [13]:
train_data.head()

,text_data,target_label
0,from : mathew mathew manti co uk subject : alt...,alt.atheism
1,from : mathew mathew manti co uk subject : alt...,alt.atheism
2,from : i dbstu rz tu - bs de benedikt rosenau ...,alt.atheism
3,from : mathew mathew manti co uk subject : re ...,alt.atheism
4,from : strom watson ibm com rob strom subject ...,alt.atheism


In [14]:
test_data.head()

,text_data,target_label
0,from : decay cbnewsj cb att com dean kaflowitz...,alt.atheism
1,from : cfaehl vesta unm edu chris faehl subjec...,alt.atheism
2,from : mathew mathew manti co uk subject : re ...,alt.atheism
3,from : dps nasa kodak com dan schaertel subjec...,alt.atheism
4,from : halat panther bear jim halat subject : ...,alt.atheism


In [15]:
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(train_data['text_data'])

sequences = tokenizer.texts_to_sequences(train_data['text_data'])
data = pad_sequences(sequences, maxlen=50)

In [16]:
print(data.shape)

(11314, 50)


In [17]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rayha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [18]:
train_data['tokenized'] = train_data.apply(lambda row : nltk.word_tokenize(row['text_data']), axis=1)

In [19]:
train_data.head()

,text_data,target_label,tokenized
0,from : mathew mathew manti co uk subject : alt...,alt.atheism,"[from, :, mathew, mathew, manti, co, uk, subje..."
1,from : mathew mathew manti co uk subject : alt...,alt.atheism,"[from, :, mathew, mathew, manti, co, uk, subje..."
2,from : i dbstu rz tu - bs de benedikt rosenau ...,alt.atheism,"[from, :, i, dbstu, rz, tu, -, bs, de, benedik..."
3,from : mathew mathew manti co uk subject : re ...,alt.atheism,"[from, :, mathew, mathew, manti, co, uk, subje..."
4,from : strom watson ibm com rob strom subject ...,alt.atheism,"[from, :, strom, watson, ibm, com, rob, strom,..."


In [20]:
model_w2v = Word2Vec(train_data['tokenized'], size=100 , min_count=1)

In [21]:
def read_my_glove_vecs(word_to_vect_model):
    
    X = word_to_vect_model.wv[model_w2v.wv.vocab]
    
    vocab_words = []
    for k, v in model_w2v.wv.vocab.items():
        vocab_words.append(k)
        
    my_embeddings_index = dict()
    my_word_to_index = dict()
    my_index_to_word = dict()
 
    for i in range(len(vocab_words)):
        my_embeddings_index[vocab_words[i]] = X[i]
        my_word_to_index[vocab_words[i]] = i+1
        my_index_to_word[i+1] = vocab_words[i]
        
    return my_word_to_index, my_index_to_word, my_embeddings_index

In [22]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

In [23]:
def label_to_emoji(label):
    """
    Converts a label (int or string) into the corresponding emoji code (string) ready to be printed
    """
    return emoji_dictionary[str(label)]

In [24]:
Y_oh_train = convert_to_one_hot(Y_train, C = 20)
Y_oh_test = convert_to_one_hot(Y_test, C = 20)
Y_oh_test[-5]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  1.])

Let's see what `convert_to_one_hot()` did. Feel free to change `index` to print out different values. 

In [25]:
index = 5
print(Y_train[index], "is converted into one hot", Y_oh_train[index])

0 is converted into one hot [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]


In [26]:
word_to_index, index_to_word, word_to_vec_map = read_my_glove_vecs(model_w2v)

In [27]:
len(word_to_vec_map["the"])

100

In [28]:
word_to_index["^"]

1900

All the data is now ready to be fed into the Emojify-V1 model. Let's implement the model!

In [29]:
def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4). 
    
    Arguments:
    X -- array of sentences (strings), of shape (m, 1)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this. 
    
    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """
    
    m = X.shape[0]                                   # number of training examples
    
    ### START CODE HERE ###
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m, max_len))
    
    for i in range(m):                               # loop over training examples
        
        # Convert the ith training sentence in lower case and split is into words. You should get a list of words.
        sentence_words =[i.lower() for i in X[i].split()]
        #print(sentence_words)
        
        # Initialize j to 0
        j = 0
        #print(i)
        # Loop over the words of sentence_words
        for w in sentence_words:
            # Set the (i,j)th entry of X_indices to the index of the correct word.
            #print(i)
            #print(j)
            #print(w)
            if w not in word_to_index:
                X_indices[i, j] = 0
            else:
                X_indices[i, j] = word_to_index[w]
            # Increment j to j + 1
            j = j + 1
            
    ### END CODE HERE ###
    
    return X_indices

Run the following cell to check what `sentences_to_indices()` does, and check your results.

In [30]:
X1 = np.array(["like one fgdfgdfgdfgdgd vfdsdf sfsdfsfogghgfhgfhgfhwewewe"])
X1_indices = sentences_to_indices(X1,word_to_index,max_len = 5)
print("X1 =", X1)
print("X1_indices =", X1_indices)

X1 = ['like one fgdfgdfgdfgdgd vfdsdf sfsdfsfogghgfhgfhgfhwewewe']
X1_indices = [[ 65.  66.   0.   0.   0.]]


In [31]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.
    
    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation.
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """
    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["love"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    ### START CODE HERE ###
    # Initialize the embedding matrix as a numpy array of zeros of shape (vocab_len, dimensions of word vectors = emb_dim)
    emb_matrix = np.zeros((vocab_len, emb_dim))
    #print(emb_matrix[-1])
    
    # Set each row "index" of the embedding matrix to be the word vector representation of the "index"th word of the vocabulary
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]
    
    #print(word_to_vec_map[word])
    #print("after")
    #print(emb_matrix[-1])

    # Define Keras embedding layer with the correct output/input sizes, make it trainable. Use Embedding(...). Make sure to set trainable=False. 
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    ### END CODE HERE ###

    # Build the embedding layer, it is required before setting the weights of the embedding layer. Do not modify the "None".
    embedding_layer.build((None,))
    
    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [32]:
embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
print("weights[0][1][3] =", embedding_layer.get_weights()[0][1][3])

weights[0][1][3] = -2.5847


In [33]:
def Classification_Model(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.
    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """
    
    ### START CODE HERE ###
    # Define sentence_indices as the input of the graph, it should be of shape input_shape and dtype 'int32' (as it contains indices).
    sentence_indices = Input(input_shape, dtype='int32')
    
    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    
    # Propagate sentence_indices through your embedding layer, you get back the embeddings
    embeddings = embedding_layer(sentence_indices)    
    
    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a batch of sequences.
    X = LSTM(128, return_sequences=True)(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # Be careful, the returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(128, return_sequences=False)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer with softmax activation to get back a batch of 5-dimensional vectors.
    X = Dense(20)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices, outputs=X)
    
    ### END CODE HERE ###
    
    return model

In [34]:
maxLen = len(max(train_data["text_data"], key=len).split())
maxLen

24388

In [35]:
model = Classification_Model((maxLen,), word_to_vec_map, word_to_index)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 24388)             0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 24388, 100)        7576700   
_________________________________________________________________
lstm_1 (LSTM)                (None, 24388, 128)        117248    
_________________________________________________________________
dropout_1 (Dropout)          (None, 24388, 128)        0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                2580      
__________

As usual, after creating your model in Keras, you need to compile it and define what loss, optimizer and metrics your are want to use. Compile your model using `categorical_crossentropy` loss, `adam` optimizer and `['accuracy']` metrics:

In [36]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [37]:
X_train = np.array(train_data["text_data"])
X_test = np.array(test_data["text_data"])

In [38]:
maxLenTrain = len(max(train_data["text_data"], key=len).split())
maxLenTrain

24388

In [39]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLenTrain)
Y_train_oh = convert_to_one_hot(Y_train, C = 20)

In [40]:
print(len(X_train_indices[1]))

24388


In [41]:
X_train_indices[1]

array([ 1.,  2.,  3., ...,  0.,  0.,  0.])

In [42]:
maxLenTest = len(max(test_data["text_data"], key=len).split())
maxLenTest

45885

In [43]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLenTest)
Y_test_oh = convert_to_one_hot(Y_test, C = 20)

In [ ]:
X_test_indices[3]

array([  1.00000000e+00,   2.00000000e+00,   1.32710000e+04, ...,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00])

Fit the Keras model on `X_train_indices` and `Y_train_oh`. We will use `epochs = 50` and `batch_size = 32`.

In [ ]:
model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)

Epoch 1/50


Your model should perform close to **95% accuracy** on the training set. The exact accuracy you get may be a little different. Run the following cell to evaluate your model on the test set. 

In [ ]:
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Test accuracy = ", acc)

You should get a test accuracy between 80% and 95%. Run the cell below to see the mislabelled examples. 

Now you can try it on your own example. Write your own sentence below. 